In [ ]:
# Import some basic libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('paper')
import requests
import os
def download(url, local_filename=None):
    """
    Downloads the file in the ``url`` and saves it in the current working directory.
    """
    data = requests.get(url)
    if local_filename is None:
        local_filename = os.path.basename(url)
    with open(local_filename, 'wb') as fd:
        fd.write(data.content)

# Hands-on Activity 16.1: Logistic regression with one variable

## Objectives

+ To demonstrate logistic regression with one variable

## High melting explosives sensitivity

[High Melting Explosives](https://en.wikipedia.org/wiki/HMX) (HMX) have applications as detonators of nuclear weapons and as solid rocket propellants.
We will use logistic regression to build the probability that a specific HMX block explodes when dropped from a given height.
To this end, we will use data from a 1987 Los Alamos Report
(L. Smith, “Los Alamos National Laboratory explosives orientation course: Sensitivity and sensitivity tests to impact, friction, spark and shock,” Los Alamos National Lab, NM (USA), Tech. Rep., 1987).
Let's download the raw data and load them.
We will use the [Python Data Analysis Library](https://pandas.pydata.org/):

In [ ]:
# Download the data file:
url = 'https://raw.githubusercontent.com/PredictiveScienceLab/data-analytics-se/master/activities/hmx_data.csv'
download(url)
# Load the data using pandas
import pandas as pd
data = pd.read_csv('hmx_data.csv')

Each row of the data is a different experiment.
There are two columns:

+ First column is **Height**: From what height (in cm) was the specimen dropped from.
+ Second column is **Result**: Did the specimen explode (E) or not (N)?

Here is how to see the raw data:

In [ ]:
data

Let's encode the labels as $1$ and and $0$ instead of E and N.
Let's do this below:

In [ ]:
# Extract data for regression
# Heights as a numpy array
x = data['Height'].values
# The labels must be 0 and 1
# We will use a dictionary to indicate our labeling
label_coding = {'E': 1, 'N': 0}
y = np.array([label_coding[r] for r in data['Result']])
data['y'] = y
data

Let's visualize the data.
Notice that lots of observations fall on top of each other.

In [ ]:
fig, ax = plt.subplots(dpi=150)
ax.plot(x, y, 'o')
ax.set_xlabel('$x$ (Height in cm)')
ax.set_ylabel('Result ($0=N; 1=E$)');

Let's train a logistic regression model with just a linear feature using scikit-learn:

In [ ]:
from sklearn.linear_model import LogisticRegression
# Make the design matrix
X = np.hstack([np.ones((x.shape[0], 1)), x[:, None]])
# Train the model (penalty = 'none' means that we do not add a prior on the weights)
# we are effectively just maximizing the likelihood of the data
model = LogisticRegression(penalty='none').fit(X, y)

Here is how you can get the trained weights of the model:

In [ ]:
model.coef_

And here is how you can make predictions at some arbitrary heights:

In [ ]:
x_predict = np.array([10.0, 20.0, 30.0, 40.0, 50.0])
X_predict = np.hstack([np.ones((x_predict.shape[0], 1)), x_predict[:, None]])
predictions = model.predict_proba(X_predict)
predictions

Note that the model gave us back the probability of each class.
If you wanted, you could ask for the class of maximum probability for each prediction input:

In [ ]:
model.predict(X_predict)

To visualize the predictions of the model as a function of the height, we can do this:

In [ ]:
fig, ax = plt.subplots(dpi=150)
xx = np.linspace(20.0, 45.0, 100)
XX = np.hstack([np.ones((xx.shape[0], 1)), xx[:, None]])
predictions_xx = model.predict_proba(XX)
ax.plot(xx, predictions_xx[:, 0], label='Probability of N')
ax.plot(xx, predictions_xx[:, 1], label='Probability of E')
ax.set_xlabel('$x$ (cm)')
ax.set_ylabel('Probability')
plt.legend(loc='best');

### Questions

+ What is the probability of explosition when the height becomes very small?
+ What is the probability of explosition when the height becomes very large?
+ At what height it is particularly difficult to predict what is going to happen?